# rectification

In [1]:
import glob
import json
from multiprocessing import Pool, pool

import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# load the parameters
params = json.load(open('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/stereo_parameters.json'))

cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
               params['CameraParameters1']['TangentialDistortion'] + \
               [params['CameraParameters1']['RadialDistortion'][2]]
distCoeffs1 = np.array(distCoeffs1)

distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
               params['CameraParameters2']['TangentialDistortion'] + \
               [params['CameraParameters2']['RadialDistortion'][2]]
distCoeffs2 = np.array(distCoeffs2)

R = np.array(params['RotationOfCamera2']).transpose()
T = np.array(params['TranslationOfCamera2']).transpose()

In [3]:
imageSize = (4096, 3000)

In [4]:
# rectification
(R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

In [5]:
left_paths = sorted(glob.glob('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001/left*'))
right_paths = sorted(glob.glob('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001/right*'))

In [6]:
def remap_and_save(img_path):
    exp_number = "181001010001"
    img = cv2.imread(img_path)
    img_remap = cv2.remap(img, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    new_path = img_path.replace(exp_number, exp_number + '_rectified')
    print(new_path)
    cv2.imwrite(new_path, img)

In [7]:
# multiproceesing
p = Pool(6)

/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475660161.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475718163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475753164.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475735163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475677161.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475697162.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475661161.jpg
/root/data/rnd/small_pen_data_coll

/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475745163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475670161.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475709163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475688162.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475763167.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475746163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475728163.jpg
/root/data/rnd/small_pen_data_coll

/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475825165.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475790165.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475863168.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475808165.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475774164.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475846166.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475826165.jpg
/root/data/rnd/small_pen_data_coll

/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475839166.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475783165.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475800164.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475818166.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475857166.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475660161.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/left_sotra-small-pen_0_1538475840166.jpg
/root/data/rnd/small_pen_data_col

/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475727163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475692163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475670161.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475711162.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475762163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475693162.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475745163.jpg
/root/data/rnd/small_pen_da

/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475721163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475737163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475703162.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475754163.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475773164.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475772164.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475809166.jpg
/root/data/rnd/small_pen_da

/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475800164.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475782164.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475874167.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475856167.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475818166.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475838167.jpg
/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001_rectified/right_sotra-small-pen_0_1538475801165.jpg
/root/data/rnd/small_pen_da

In [8]:
p.map(remap_and_save, left_paths + right_paths)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# display